### Sentiment Analysis can help us decipher the mood and emotions of general public and gather insightful information regarding the context. Sentiment Analysis is a process of analyzing data and classifying it based on the need of the research 

In [ ]:
import requests

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
r = requests.get('https://www.yelp.com/biz/tesla-san-francisco?osq=Tesla+Dealership')
# you can choose any website

In [ ]:
r.status_code

In [ ]:
r.text

In [ ]:
soup = BeautifulSoup(r.text, 'html.parser')

In [ ]:
divs = soup.findAll(class_="comment__09f24__gu0rG css-1sufhje")
# choose the class which is common to the content of interest

In [ ]:
reviews = []
for div in divs:
    reviews.append(div.text)

In [ ]:
reviews

## Analysing the data

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.DataFrame(np.array(reviews), columns = ['review'])

In [ ]:
df.head()

In [ ]:
len(df['review'])

In [ ]:
df['word_count']=df['review'].apply(lambda x : len(x.split()))

In [ ]:
df.head()

In [ ]:
df['char_count']=df['review'].apply(lambda x : len(x))

In [ ]:
df.head()

In [ ]:
def average_words(x):
    words = x.split()
    return sum(len(word) for word in words)/ len(words)

In [ ]:
df['avereage_word_length']= df['review'].apply(lambda x: average_words(x))

In [ ]:
df.head()

In [ ]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

In [ ]:
stop_words = stopwords.words('english')

In [ ]:
df['stopword_count'] = df['review'].apply(lambda x : len([word for word in x.split() if word.lower() in stop_words]))

In [ ]:
df.head()

In [ ]:
df['stopword_rate']= df['stopword_count']/df['word_count']

In [ ]:
df.head()

In [ ]:
df.sort_values(by = 'stopword_rate')

In [ ]:
df.describe()

## Data Cleaning

In [ ]:
df['lowercase'] =  df['review'].apply(lambda x : " ".join(word.lower() for word in x.split()))

In [ ]:
df.head()

In [ ]:
df['punctuation']  = df['lowercase'].str.replace('[^\w\s]', '')

In [ ]:
df.head()

In [ ]:
df["stopwords"] = df['punctuation'].apply(lambda x : " ".join(word for word in x.split() if word not in stop_words))

In [ ]:
df.head()

In [ ]:
pd.Series(" ".join(df['stopwords']).split()).value_counts()[:50]

In [ ]:
other_stop_words = ['get','also','us','new']

In [ ]:
len(other_stop_words)

In [ ]:
df.head()

In [ ]:
df['cleanreview']  = df['stopwords'].apply(lambda x : " ".join(word for word in x.split() if word not in other_stop_words))

In [ ]:
df.head()

##  Lemmatization

In [ ]:
from textblob import Word
nltk.download('omw-1.4')

In [ ]:
df['lemmatize']=df['cleanreview'].apply(lambda x :" ".join(Word(word).lemmatize() for word in x.split()))

In [ ]:
df.head()

##  Sentiment Analysis

### TextBlob returns polarity and subjectivity of a sentence. Polarity lies between [-1,1], -1 defines a negative sentiment and 1 defines a positive sentiment. Negation words reverse the polarity. TextBlob has semantic labels that help with fine-grained analysis. For example — emoticons, exclamation mark, emojis, etc. Subjectivity lies between [0,1]. Subjectivity quantifies the amount of personal opinion and factual information contained in the text. The higher subjectivity means that the text contains personal opinion rather than factual information. 

In [ ]:
df['clean_review_word_count'] = df['cleanreview'].apply(lambda x : len(x.split()))

In [ ]:
from textblob import TextBlob

In [ ]:
df['clean_rate']=df['clean_review_word_count']/df['word_count']

In [ ]:
df.head()

In [ ]:
df['polarity'] = df['lemmatize'].apply(lambda x: TextBlob(x).sentiment[0])

In [ ]:
df['subjectivity'] = df['lemmatize'].apply(lambda x: TextBlob(x).sentiment[1])

In [ ]:
df.head()

In [ ]:
df.sort_values(by='polarity')